In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
import sys
import json
import glob
import numpy as np
from collections import defaultdict, namedtuple
from pprint import pprint
from copy import copy

In [ ]:
from path_guiding_kdtree_loader import read_records, convert_data

In [ ]:
from matplotlib import pyplot
import matplotlib
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def pdf_image(ax, gmm):
    t = np.linspace(-1.,1., 100)
    x,y = np.meshgrid(t,t)
    coord_list = np.vstack((x.ravel(), y.ravel())).T
    pdf_vals = gmm.pdf(coord_list)
    pdf_vals = np.reshape(pdf_vals, (t.size,t.size))
    return ax.imshow(pdf_vals[::-1,::], extent=(-1.,1,-1.,1.))

def rot(a):
    a = np.pi/180*a
    cs = np.cos(a)
    sn = np.sin(a)
    rot = [
        [cs, sn],
        [-sn,cs]
    ]
    return np.matrix(rot)

In [ ]:
def run_fit(cell):
    gmm = path_guiding.GMM2d()
    init_uniform_means_on_cicle = np.array([
        [-0.02081824, -0.00203204],
        [ 0.4037143 , -0.5633242 ],
        [-0.60468113,  0.32800356],
        [-0.12180968,  0.6798401 ],
        [ 0.6943762 , -0.03451705],
        [ 0.4580511 ,  0.52144015],
        [-0.63349193, -0.26706135],
        [-0.18766472, -0.66355205]
    ])
    sigma_inv = np.full(8, 10.)
    init_precisions = np.zeros((8,2,2), dtype=np.float32)
    init_precisions[:,0,0] = sigma_inv
    init_precisions[:,1,1] = sigma_inv
    init_weights = np.full(8, 1./8., dtype=np.float32)
    gmm.initialize(init_weights, init_uniform_means_on_cicle, init_precisions)
    prior_nu = 1.00001; prior_alpha = 2.01; prior_u = 1.e-5; max_iters = 10;
    
    xs = cell.proj
    ws = np.average(cell.vals,axis=1)
    gmm.fit(xs, ws, prior_nu, prior_alpha, prior_u, max_iters)
    return gmm

def incremental_fit(cell):
    gmm = path_guiding.GMM2d()
    init_uniform_means_on_cicle = np.array([
        [-0.02081824, -0.00203204],
        [ 0.4037143 , -0.5633242 ],
        [-0.60468113,  0.32800356],
        [-0.12180968,  0.6798401 ],
        [ 0.6943762 , -0.03451705],
        [ 0.4580511 ,  0.52144015],
        [-0.63349193, -0.26706135],
        [-0.18766472, -0.66355205]
    ])
    sigma_inv = np.full(8, 10.)
    init_precisions = np.zeros((8,2,2), dtype=np.float32)
    init_precisions[:,0,0] = sigma_inv
    init_precisions[:,1,1] = sigma_inv
    init_weights = np.full(8, 1./8., dtype=np.float32)
    gmm.initialize(init_weights, init_uniform_means_on_cicle, init_precisions)
    
    prior_nu = 100; prior_alpha = 2.1; prior_u = 10; maximization_step_every = 100;
    
    incremental = path_guiding.GMM2dFitIncremental(
        prior_nu = prior_nu, 
        prior_alpha = prior_alpha, 
        prior_u = prior_u, 
        maximization_step_every = maximization_step_every)
    
    xs = cell.proj
    ws = np.average(cell.vals,axis=1)
    incremental.fit(gmm, xs, ws)
    
    return gmm

In [ ]:
#PREFIX="D:\\tmp\\"
PREFIX="/tmp/"

In [ ]:
for i in range(11):
    tree, celldata = convert_data(read_records(PREFIX+f'radiance_records_{i}.json'), load_samples = False)
    weights = np.array([cd['average_weight'] for cd in celldata])
    print (f"{i}: {len(celldata)}, average weight = {np.average(weights)}")
    pyplot.bar(x = np.arange(len(weights)), height=weights)
    pyplot.show()

In [ ]:
#for i in range(14):
#    tree, celldata = convert_data(read_records(PREFIX+f'radiance_records_{i}.json'), load_samples = False)

In [ ]:
#%matplotlib notebook

In [ ]:
tree, celldata = convert_data(read_records(PREFIX+'radiance_records_14.json'), load_samples = False, build_boxes = True)

In [ ]:
n = [c.num_points for c in celldata]
#pyplot.plot(n)
sum(n)# / (1280*960)

In [ ]:
coords = np.concatenate([c.center[np.newaxis,:] for c in celldata], axis=0)
mask_roi = (coords[:,1] < 0.5) & (coords[:,1] > 0.0) & (coords[:,0] < 0.25) & (coords[:,0] > -0.25) & (coords[:,2] < 0.25) & (coords[:,2] > -0.25)
print (np.where(mask_roi))

In [ ]:
def plot_vmf_pdf(ax, vmfm):
    # https://scipython.com/book/chapter-8-scipy/examples/visualizing-the-spherical-harmonics/
    phi = np.linspace(0, np.pi, 100)
    theta = np.linspace(0, 2*np.pi, 100)
    phi, theta = np.meshgrid(phi, theta)

    # The Cartesian coordinates of the unit sphere
    x = np.sin(phi) * np.cos(theta)
    y = np.sin(phi) * np.sin(theta)
    z = np.cos(phi)

    pts = np.vstack((x.ravel(),y.ravel(),z.ravel())).T
    fcolors = vmfm.pdf(pts).reshape(x.shape)
    fcolors /= fcolors.max()
    
    ax.plot_surface(x, y, z,  rstride=1, cstride=1, facecolors=matplotlib.cm.jet(fcolors))

In [ ]:
%matplotlib notebook

In [ ]:
# For gaussian mixture
data_selection = [celldata[int(i)] for i in [73, 110, 164, 165]]
for cell1 in data_selection:
    if 0:
        fig, axes = pyplot.subplots(1,3, figsize=(15,5))
        fig.suptitle(f"#samples = {cell1.num_points}")
        if cell1.val is not None:
            w = np.average(cell1.val,axis=1) + 1.e-9
            norm = np.average(w)
            w /= norm
            axes[0].scatter(*cell1.proj.T, c = w, s=w, alpha = 0.2, edgecolor='none')
            axes[0].add_artist(pyplot.Circle((0, 0), 1, fill=False, color = 'k'))
        pdf_image(axes[1], cell1.mixture_learned)
        axes[1].set(title = 'learned')
        pdf_image(axes[2], cell1.mixture_sampled)
        axes[2].set(title = 'sampled')
        pyplot.show()
    if 1:
        fig = pyplot.figure()
        ax = fig.add_subplot(121, projection='3d')
        plot_vmf_pdf(ax, cell1.mixture_learned)
        fig = pyplot.figure()
        ax = fig.add_subplot(122, projection='3d')
        plot_vmf_pdf(ax, cell1.mixture_sampled)
        pyplot.show()